In [1]:
import numpy as np

from lets_plot import *
import pandas as pd
import vega_datasets
import seaborn as sns
LetsPlot.setup_html()

In [2]:
data = sns.load_dataset("penguins").dropna()
data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


## Notation reminder

A **random variable** $X$ represents an unknown *random* value. The **pdf/pmf** for a random variable is the function that specifies the probability or probability density at any given outcome.

$$p(x) = Prob\left(X =x \right)$$

# Answering questions about multi-variable datasets

### Simple case: take one variable

In [3]:
ggplot() + geom_histogram(data=data, mapping=aes(x='body_mass_g')) + ggsize(800, 300)

What is the *average* mass of a penguin?

In [4]:
data['body_mass_g'].mean()

np.float64(4207.057057057057)

In [5]:
ggplot() + geom_histogram(data=data, mapping=aes(x='body_mass_g')) + \
    geom_vline(xintercept=data['body_mass_g'].mean(), color='red') + ggsize(800, 300)

What *fraction* of penguins have a mass over 5kg?

- If we treat this as a population estimate, we could interpret this as the *probability* that a penguin has a mass over 5kg.

$$ Prob\left[ \text{mass} > 5000 \right]

In [6]:
(data['body_mass_g'] >= 5000).mean()

np.float64(0.2012012012012012)

In [7]:
mass_data['Mass over 5kg']

NameError: name 'mass_data' is not defined

In [8]:
mass_data = data.copy()
mass_data['Mass over 5kg'] = data['body_mass_g'] > 5000
ggplot() + geom_histogram(data=mass_data, mapping=aes(x='body_mass_g', fill='Mass over 5kg')) + ggsize(800, 300)

In [9]:
data.query('(body_mass_g >= 4000) & (body_mass_g <= 5000)').shape[0] / data.shape[0]

0.3333333333333333

In [10]:
mass_data = data.copy()
mass_data['Mass in 4-5kg'] = (data['body_mass_g'] >= 4000) & (data['body_mass_g'] <= 5000)
ggplot() + geom_histogram(data=mass_data, mapping=aes(x='body_mass_g', fill='Mass in 4-5kg')) + ggsize(800, 300)

What fraction are the *chinstrap* species?

In [11]:
(data['species'] == 'Chinstrap').mean()

np.float64(0.2042042042042042)

In [12]:
ggplot() + geom_bar(data=data, mapping=aes(x='species', fill='species'))

In [13]:
ggplot() + geom_bar(data=data, mapping=aes(fill='species', y='..prop..'))

Here we are asking questions about the distribution of a *single* variable $p(x)$ out of a larger group (e.g. $\{x, y\}$). We call this a **marginal** distribution.

### Questions involving > 1 variable

Distinguish **univariate** and **multivariate** statistics.

#### Univariate: statistics that result in a single value

What fraction of penguins are **male** *and* **gentoo**?

In [14]:
((data['species'] == 'Gentoo') & (data['sex'] == 'Male')).mean()

np.float64(0.1831831831831832)

$$Prob[\text{species} = \text{Gentoo } \text{ and } \text{ sex } = \text{Male}]

In [15]:
data_spsx = data[['species', 'sex']].copy()
data_spsx['count'] = 1
data_spsx

,species,sex,count
0,Adelie,Male,1
1,Adelie,Female,1
2,Adelie,Female,1
4,Adelie,Female,1
5,Adelie,Male,1
...,...,...,...
338,Gentoo,Female,1
340,Gentoo,Female,1
341,Gentoo,Male,1
342,Gentoo,Female,1


In [16]:
data_spsx = data[['species', 'sex']].copy()
data_spsx['count'] = 1
data_spsx = data_spsx.groupby(['species', 'sex']).count().reset_index()
data_spsx

,species,sex,count
0,Adelie,Female,73
1,Adelie,Male,73
2,Chinstrap,Female,34
3,Chinstrap,Male,34
4,Gentoo,Female,58
5,Gentoo,Male,61


In [17]:
ggplot() + geom_tile(data=data_spsx, mapping=aes(x='species', y='sex', fill='count')) + \
    geom_text(data=data_spsx, mapping=aes(x='species', y='sex', label='count'), color='white')

In [18]:
data_spsx.pivot(index='species', columns='sex', values='count')

sex,Female,Male
species,,
Adelie,73,73
Chinstrap,34,34
Gentoo,58,61


In [19]:
ggplot() + geom_point(data=data, mapping=aes(x='flipper_length_mm', y='bill_length_mm'))

In [20]:
data_len = data.copy()
data_len['Long bill, short flipper'] = (data_len['flipper_length_mm'] < 200) & (data_len['bill_length_mm'] > 40)
data_len['Long bill, short flipper'].mean()

np.float64(0.2702702702702703)

In [21]:
ggplot() + geom_point(data=data_len, mapping=aes(x='flipper_length_mm', y='bill_length_mm', color='Long bill, short flipper'))

What is the **correlation** between bill and flipper length?

In [22]:
data[['flipper_length_mm', 'bill_length_mm']].corr()

,flipper_length_mm,bill_length_mm
flipper_length_mm,1.000000,0.653096
bill_length_mm,0.653096,1.000000


In [23]:
data[['flipper_length_mm', 'bill_length_mm']].corr().loc['bill_length_mm', 'flipper_length_mm']

np.float64(0.6530956386670871)

#### Multivariate: statistics that result in multiple values

What is the **mean** of bill length and flipper length?

In [24]:
data[['flipper_length_mm', 'bill_length_mm']].mean().to_frame().T

,flipper_length_mm,bill_length_mm
0,200.966967,43.992793


In [25]:
ggplot() + geom_point(data=data, mapping=aes(x='flipper_length_mm', y='bill_length_mm')) + \
    geom_point(data=data[['flipper_length_mm', 'bill_length_mm']].mean().to_frame().T, 
               mapping=aes(x='flipper_length_mm', y='bill_length_mm'), color='red', size=5)

Here we are asking questions about the distribution of a *a set of variables* $p(x, y)$ (e.g. $\{x, y\}$). We call this a **joint** distribution between $x$ and $y$.

### Questions involving a **subset** of the data



Assuming we *know* something about the value of one of our variables, what can we say about the others?

What fraction of *Adelie* Penguins live on *Biscoe* island?

Or as an *estimated* probability:
$$Prob\left[ \text{island} = \text{Biscoe}  \mid  \text{species} = \text{Adelie} \right]$$

In [26]:
data.query('species == "Adelie"').groupby('island').count()[['species']]

,species
island,
Biscoe,44
Dream,55
Torgersen,47


In [27]:
(data.query('species == "Adelie"')['island'] == 'Biscoe').mean()

np.float64(0.3013698630136986)

In [28]:
ggplot() + geom_bar(data=(data.query('species == "Adelie"')), mapping=aes(x='island', fill='island'))

In [29]:
ggplot() + geom_bar(data=data, mapping=aes(x='species', fill='island'))

In [30]:
ggplot() + geom_bar(data=data, mapping=aes(x='island', fill='island')) + facet_grid(x='species')

What fraction of *Adelie* penguins have a mass >= 4kg?

In [31]:
(data.query('species == "Adelie"')['body_mass_g'] >= 4000).mean()

np.float64(0.2602739726027397)

In [32]:
adelie_mass_data = data.query('species == "Adelie"')
adelie_mass_data['Mass over 4kg'] = adelie_mass_data['body_mass_g'] > 4000
ggplot() + geom_histogram(data=adelie_mass_data, mapping=aes(x='body_mass_g', fill='Mass over 4kg')) + ggsize(800, 300)

/var/folders/ct/cvh0qs6j5rlbcp004rxwtwh80000gn/T/ipykernel_11684/4119500255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adelie_mass_data['Mass over 4kg'] = adelie_mass_data['body_mass_g'] > 4000


In [33]:
ggplot() + geom_histogram(data=data, mapping=aes(x='body_mass_g', fill='species')) + ggsize(800, 300)

In [284]:
ggplot() + geom_histogram(data=data, mapping=aes(x='body_mass_g', fill='species')) + facet_grid(x='species') + ggsize(800, 300) 

In [34]:
ggplot() + geom_histogram(data=data.query('species == "Chinstrap"'), mapping=aes(x='body_mass_g', fill='species', y='..sumprop..'), alpha=0.2)\
    + geom_histogram(data=data.query('species == "Adelie"'), mapping=aes(x='body_mass_g', fill='species', y='..sumprop..'), alpha=0.2)\
     + geom_histogram(data=data.query('species == "Gentoo"'), mapping=aes(x='body_mass_g', fill='species', y='..sumprop..'), alpha=0.2)\
     + ggsize(800, 300)

In [ ]:
ggplot() + geom_histogram(data=data.query('species == "Chinstrap"'), mapping=aes(x='body_mass_g', fill='species', y='..sumprop..'), alpha=0.2)\
    + geom_histogram(data=data.query('species == "Adelie"'), mapping=aes(x='body_mass_g', fill='species', y='..sumprop..'), alpha=0.2)\
     + geom_histogram(data=data.query('species == "Gentoo"'), mapping=aes(x='body_mass_g', fill='species', y='..sumprop..'), alpha=0.2)\
     + ggsize(800, 300)

In [37]:
ggplot() + geom_histogram(data=data.query('species == "Chinstrap"'), mapping=aes(x='bill_depth_mm', fill='species', y='..sumprop..'), alpha=0.2)\
    + geom_histogram(data=data.query('species == "Adelie"'), mapping=aes(x='bill_depth_mm', fill='species', y='..sumprop..'), alpha=0.2)\
     + geom_histogram(data=data.query('species == "Gentoo"'), mapping=aes(x='bill_depth_mm', fill='species', y='..sumprop..'), alpha=0.2)\
     + ggsize(800, 300)

In [285]:
ggplot() + geom_histogram(data=data, mapping=aes(x='body_mass_g', y='..sumprop..', fill='species')) + facet_grid(x='species') + ggsize(800, 300) 

# Bootstrapping

Estimates the **uncertainty** in a statistic (like model-based statistics)

A simpler bootstrap sample:

In [334]:
data.sample(frac=1., replace=True)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
332,Gentoo,Biscoe,43.5,15.2,213.0,4650.0,Female
95,Adelie,Dream,40.8,18.9,208.0,4300.0,Male
49,Adelie,Dream,42.3,21.2,191.0,4150.0,Male
135,Adelie,Dream,41.1,17.5,190.0,3900.0,Male
72,Adelie,Torgersen,39.6,17.2,196.0,3550.0,Female
...,...,...,...,...,...,...,...
251,Gentoo,Biscoe,42.8,14.2,209.0,4700.0,Female
303,Gentoo,Biscoe,50.0,15.9,224.0,5350.0,Male
200,Chinstrap,Dream,51.5,18.7,187.0,3250.0,Male
21,Adelie,Biscoe,37.7,18.7,180.0,3600.0,Male


In [335]:
import math 

def bootstrap_distribution(data, statistic_fun, nsamples):
    estimates = []                       # Distribution of statistic estimates
    for i in range(nsamples):            # Resample many times
        sample = data.sample(frac=1., replace=True)  # Get a new bootstrapped sample
        stat = statistic_fun(sample)     # Compute our statistic
        estimates.append(stat)           # add it to the distribution
    return estimates

def simple_simulation_ci(data, statistic_fun, nsamples, confidence=0.95):
    # Distribution of statistic estimates
    estimates = sorted(bootstrap_distribution(data, statistic_fun, nsamples))
    margin = (1 - confidence) / 2
    lower = estimates[math.ceil(nsamples * margin)]
    upper = estimates[math.ceil(nsamples * (1 - margin))] 
    return estimates, lower, upper

In [288]:
def statistic(data):
    return ((data['species'] == 'Gentoo') & (data['sex'] == 'Male')).mean()

estimates, lower, upper = simple_simulation_ci(data, statistic, 10000)
estimates = pd.DataFrame({"Simulated fraction of male Gentoo penguins": estimates})
ggplot() + geom_bar(data=estimates, mapping=aes(x="Simulated fraction of male Gentoo penguins")) + geom_vline(xintercept=lower, color='red', linetype='longdash') + geom_vline(xintercept=upper, color='red', linetype='longdash')


In [289]:
def statistic(data):
    return ((data['flipper_length_mm'] < 200) & (data['bill_length_mm'] > 40)).mean()

estimates, lower, upper = simple_simulation_ci(data, statistic, 10000)
estimates = pd.DataFrame({"Simulated fraction of short-flipper, long-bill penguins": estimates})
ggplot() + geom_bar(data=estimates, mapping=aes(x="Simulated fraction of short-flipper, long-bill penguins")) + geom_vline(xintercept=lower, color='red', linetype='longdash') + geom_vline(xintercept=upper, color='red', linetype='longdash')


In [290]:
def statistic(data):
    return (data['body_mass_g'] >= 4000).mean()

estimates, lower, upper = simple_simulation_ci(data.query('species == "Adelie"'), statistic, 10000)
estimates = pd.DataFrame({"Simulated fraction of Adelie penguins over 4kg": estimates})
ggplot() + geom_bar(data=estimates, mapping=aes(x="Simulated fraction of Adelie penguins over 4kg")) + geom_vline(xintercept=lower, color='red', linetype='longdash') + geom_vline(xintercept=upper, color='red', linetype='longdash')


What about the mean?

In [291]:
data[['flipper_length_mm', 'bill_length_mm']].mean().to_frame().T

,flipper_length_mm,bill_length_mm
0,200.966967,43.992793


In [336]:
def statistic(data):
    return data[['flipper_length_mm', 'bill_length_mm']].mean().to_frame().T

estimates = bootstrap_distribution(data, statistic, 10000)
estimates = pd.concat(estimates)
estimates

,flipper_length_mm,bill_length_mm
0,200.387387,43.948048
0,200.696697,44.006907
0,200.408408,44.141441
0,201.573574,43.987688
0,199.810811,43.562763
...,...,...
0,201.408408,44.097598
0,199.882883,44.188288
0,200.360360,43.819820
0,200.882883,43.963363


In [293]:
ggplot() + geom_point(data= estimates, mapping=aes(x='flipper_length_mm', y='bill_length_mm'))

In [294]:
ggplot() + geom_density2df(data= estimates, mapping=aes(x='flipper_length_mm', y='bill_length_mm', fill='..level..'), bins=20) + scale_fill_cmapmpl('Spectral', trans='reverse')

In [295]:
def flipper_statistic(data):
    return data['flipper_length_mm'].mean()

def bill_statistic(data):
    return data['bill_length_mm'].mean()

_, lower_flipper_ci, upper_flipper_ci = simple_simulation_ci(data, flipper_statistic, 10000)
_, lower_bill_ci, upper_bill_ci = simple_simulation_ci(data, bill_statistic, 10000)

In [296]:
ggplot() + geom_density2df(data= estimates, mapping=aes(x='flipper_length_mm', y='bill_length_mm', fill='..level..'), bins=20) + scale_fill_cmapmpl('Spectral', trans='reverse') +\
    geom_vline(xintercept=lower_flipper_ci, color='red') + geom_vline(xintercept=upper_flipper_ci, color='red') +\
    geom_hline(yintercept=lower_bill_ci, color='red') + geom_hline(yintercept=upper_bill_ci, color='red')

In [297]:
in_region = (estimates['bill_length_mm'] >= lower_bill_ci) & (estimates['flipper_length_mm'] >= lower_flipper_ci) & \
    (estimates['bill_length_mm'] < upper_bill_ci) & (estimates['flipper_length_mm'] < upper_flipper_ci)

in_region.mean()

np.float64(0.9139)

In [337]:
def flipper_statistic(data):
    return data['flipper_length_mm'].mean()

def bill_statistic(data):
    return data['bill_length_mm'].mean()

_, lower_flipper_ci, upper_flipper_ci = simple_simulation_ci(data, flipper_statistic, 10000, confidence=0.975)
_, lower_bill_ci, upper_bill_ci = simple_simulation_ci(data, bill_statistic, 10000, confidence=0.975)

In [338]:
ggplot() + geom_density2df(data= estimates, mapping=aes(x='flipper_length_mm', y='bill_length_mm', fill='..level..'), bins=20) + scale_fill_cmapmpl('Spectral', trans='reverse') +\
    geom_vline(xintercept=lower_flipper_ci, color='red') + geom_vline(xintercept=upper_flipper_ci, color='red') +\
    geom_hline(yintercept=lower_bill_ci, color='red') + geom_hline(yintercept=upper_bill_ci, color='red')

In [300]:
in_region = (estimates['bill_length_mm'] >= lower_bill_ci) & (estimates['flipper_length_mm'] >= lower_flipper_ci) & \
    (estimates['bill_length_mm'] < upper_bill_ci) & (estimates['flipper_length_mm'] < upper_flipper_ci)

in_region.mean()

np.float64(0.9559)

What is the **distribution** of these mean estimates?

Let's come back to this!

# Previous distributions

## Categorical

Distribution over a categorical (or ordinal) variable

$$p(x) = \sum_{c \in \text{categories}} p_c\cdot I(x=c)$$

MLE:

$$\hat{p}_c = \sum_{i=1}^N \frac{I(x_i=c)}{N}$$

In [301]:
probs = pd.DataFrame(dict(prob=[.4, .1, .2, .3], category=[1, 2, 3, 4]))
ggplot() + geom_bar(data=probs, mapping=aes(x='category', y='prob'), stat='identity')

## Normal

Distribution over a continuous quantitative variable (sum of random values)

$$p(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-0.5(\frac{x-\mu}{\sigma})^2}$$

MLE:

$$\hat{\mu} = \sum_{i=1}^N \frac{x_i}{N}$$
$$\hat{\sigma} = \sqrt{\sum_{i=1}^N \frac{(x_i - \hat{\mu})^2}{N}}$$

## Exponential

Distribution over a continuous quantitative variable (time to next event)

$$p(x) = \lambda e^{-\lambda x}$$

MLE: 

$$\hat{\lambda} = \frac{1}{\sum_{i=1}^N \frac{x_i}{N}}


In [302]:
from scipy.stats import expon
x = np.linspace(0., 20, 1000)
pdf = expon(scale=5).pdf(x)
probs = pd.DataFrame(dict(x=x, pdf=pdf))
ggplot() + geom_line(data=probs, mapping=aes(x='x', y='pdf'))

## Poisson

Distribution over a discrete quantitative variable (number of events in an interval)

$$p(x) = \frac{\lambda^x}{x!}e^{-\lambda}$$

MLE:

$$\hat{\lambda} = \sum_{i=1}^N \frac{x_i}{N}$$

In [303]:
from scipy.stats import poisson
x = np.arange(0, 20)
pdf = poisson(4).pmf(x)
probs = pd.DataFrame(dict(x=x, pdf=pdf))
ggplot() + geom_line(data=probs, mapping=aes(x='x', y='pdf')) + geom_point(data=probs, mapping=aes(x='x', y='pdf'))

# Joint distributions

In [304]:
data

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [305]:
data_spsx = data[['species', 'sex']].copy()
data_spsx['count'] = 1
data_spsx = data_spsx.groupby(['species', 'sex']).count().reset_index()
data_spsx['prob'] = data_spsx['count'] / data_spsx['count'].sum()
data_spsx

,species,sex,count,prob
0,Adelie,Female,73,0.219219
1,Adelie,Male,73,0.219219
2,Chinstrap,Female,34,0.102102
3,Chinstrap,Male,34,0.102102
4,Gentoo,Female,58,0.174174
5,Gentoo,Male,61,0.183183


In [306]:
data_spsx.pivot(index='species', columns='sex', values='prob')

sex,Female,Male
species,,
Adelie,0.219219,0.219219
Chinstrap,0.102102,0.102102
Gentoo,0.174174,0.183183


$$p(x) = \sum_y p(x, y)$$

In [307]:
data_spsx[['species', 'prob']].groupby('species').sum().reset_index()

,species,prob
0,Adelie,0.438438
1,Chinstrap,0.204204
2,Gentoo,0.357357


$$p(y\mid x) = \frac{p(x, y)}{p(x)}$$

In [308]:
p_x = data_spsx[['species', 'prob']].groupby('species').sum().reset_index().rename(columns={'prob': 'prob_x'})
p_xy = data_spsx.merge(p_x, on='species')
p_xy

,species,sex,count,prob,prob_x
0,Adelie,Female,73,0.219219,0.438438
1,Adelie,Male,73,0.219219,0.438438
2,Chinstrap,Female,34,0.102102,0.204204
3,Chinstrap,Male,34,0.102102,0.204204
4,Gentoo,Female,58,0.174174,0.357357
5,Gentoo,Male,61,0.183183,0.357357


In [309]:
p_xy['prob_conditional'] = p_xy['prob'] / p_xy['prob_x']
p_xy[['species', 'sex', 'prob_conditional']]

,species,sex,prob_conditional
0,Adelie,Female,0.500000
1,Adelie,Male,0.500000
2,Chinstrap,Female,0.500000
3,Chinstrap,Male,0.500000
4,Gentoo,Female,0.487395
5,Gentoo,Male,0.512605


We could also directly estimate our conditional model:

$$p(y \mid x = \text{Adelie})$$


In [310]:
adelie_data = data[['species', 'sex']].query('species == "Adelie"')
adelie_data['count'] = 1
adelie_data = adelie_data.groupby('sex').count()
adelie_data['count'] / adelie_data['count'].sum()

sex
Female    0.5
Male      0.5
Name: count, dtype: float64

In [311]:
data_spsx = data[['species', 'sex', 'island']].copy()
data_spsx['count'] = 1
data_spsx = data_spsx.groupby(['species', 'sex', 'island']).count().reset_index()
data_spsx['prob'] = data_spsx['count'] / data_spsx['count'].sum()
data_spsx

,species,sex,island,count,prob
0,Adelie,Female,Biscoe,22,0.066066
1,Adelie,Female,Dream,27,0.081081
2,Adelie,Female,Torgersen,24,0.072072
3,Adelie,Male,Biscoe,22,0.066066
4,Adelie,Male,Dream,28,0.084084
5,Adelie,Male,Torgersen,23,0.069069
6,Chinstrap,Female,Dream,34,0.102102
7,Chinstrap,Male,Dream,34,0.102102
8,Gentoo,Female,Biscoe,58,0.174174
9,Gentoo,Male,Biscoe,61,0.183183
